In [ ]:
pip install -r requirements.txt

In [ ]:
import pandas as pd
import tensorflow as tf
import sklearn
from tensorflow.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error


In [3]:
base = pd.read_csv('games.csv')
base = base.drop('Other_Sales', axis = 1)
base = base.drop('Global_Sales', axis = 1)
base = base.drop('Developer', axis = 1)
base = base.dropna(axis = 0)
base = base.drop('Name', axis = 1)

In [4]:
X = base.iloc[:, [0, 1, 2, 3, 7, 8, 9, 10, 11]].values
y_na = base.iloc[:, 4].values
y_eu = base.iloc[:, 5].values
y_jp = base.iloc[:, 6].values

In [5]:
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0, 2, 3, 8])], remainder='passthrough')
X = onehotencoder.fit_transform(X).toarray()

In [6]:
camada_entrada = Input(shape = (303,))
camada_oculta1 = Dense(units = 153, activation='relu')(camada_entrada)
camada_oculta2 = Dense(units = 153, activation='relu')(camada_oculta1)
camada_saida1 = Dense(units = 1, activation='linear')(camada_oculta2)
camada_saida2 = Dense(units = 1, activation='linear')(camada_oculta2)
camada_saida3 = Dense(units = 1, activation='linear')(camada_oculta2)

In [ ]:
regressor = Model(inputs = camada_entrada, outputs = [camada_saida1, camada_saida2, camada_saida3])
regressor.compile(optimizer='adam', loss = 'mse')
regressor.fit(X, [y_na, y_eu, y_jp], epochs=500, batch_size=100)

In [ ]:
previsao_na, previsao_eu, previsao_jp = regressor.predict(X)

In [ ]:
previsao_na, previsao_na.mean(), y_na.mean()

In [ ]:
mean_absolute_error(y_na, previsao_na)